In [1]:
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import datetime
from urllib.request import urlopen
import requests
import time

In [2]:
chromedriver_dir = r'C:\Users\LG\빅데이터 실습\chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(chromedriver_dir)

In [3]:
# 다음 사이트로 진행
url = 'https://search.daum.net/search?w=tv&q=%EC%8A%A4%ED%8A%B8%EB%A6%BF%20%EC%9A%B0%EB%A8%BC%20%ED%8C%8C%EC%9D%B4%ED%84%B0%20%EC%8B%9C%EC%B2%AD%EB%A5%A0&irt=tv-program&irk=91948&DA=TVP'

driver.get(url)
time.sleep(3)

In [4]:
# 더보기 진행
for i in range(1):
    xpath = """//*[@id="tv_rating"]/div[3]/div[2]/a"""
    driver.find_element_by_xpath(xpath).click()

In [5]:
# BeautifulSoup 실행

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
print(soup)

<html class="win chrome" lang="ko" xmlns="http://www.w3.org/1999/xhtml"><head profile="http://a9.com/-/spec/opensearch/1.1/">
<meta content="text/html;charset=utf-8" http-equiv="content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="off" name="autocomplete"/>
<meta content="always" name="referrer"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="스트릿 우먼 파이터 시청률 – Daum 검색" property="og:title"/>
<meta content="https://search.daum.net/search?w=tv&amp;q=%EC%8A%A4%ED%8A%B8%EB%A6%BF%20%EC%9A%B0%EB%A8%BC%20%ED%8C%8C%EC%9D%B4%ED%84%B0%20%EC%8B%9C%EC%B2%AD%EB%A5%A0" property="og:url"/>
<meta content="Daum 검색에서 스트릿 우먼 파이터 시청률에 대한 최신정보를 찾아보세요." property="og:description"/>
<meta content="https://search1.daumcdn.net/search/statics/common/img/og_search.png" property="og:image"/>
<meta content="다음검색" property="og:site_name"/>
<title>스트릿 우먼 파이터 시청률 – Daum 검색</title>
<link href="//search.daum.net/OpenSearch.xml" rel="search" title="Daum" type="applic

In [12]:
viewr_ratings = [] # 시청률 넣을 리스트
xpath_basic = """//*[@id="tv_rating"]/div[3]/table/tbody/tr[""" # xpath 기본

for i in np.arange(3, 12):
    tmp = str(i) + "]/td[2]" # 각 회차에 대한 시청률 얻어오기 위함
    xpath = xpath_basic + tmp # 시청률을 얻어오기 위한 xpath값 구함
    
    find_element = driver.find_element_by_xpath(xpath)
    view = find_element.get_attribute('innerHTML')
    viewr_ratings.append(view)

In [13]:
# 9회, 8회, 7회, ... , 2회, 1회 순서
viewr_ratings

['2.523%',
 '2.887%',
 '2.487%',
 '2.672%',
 '2.424%',
 '2.593%',
 '1.871%',
 '1.897%',
 '0.822%']

In [18]:
# 회차, 시청률 저장할 데이터프레임
streetwomanfiger_viewr = pd.DataFrame(columns=['회차', '시청률', '회차_merge용'])
streetwomanfiger_viewr

,회차,시청률,회차_merge용


In [19]:
# 데이터프레임에 저장
for i in np.arange(0, len(viewr_ratings)):
    episode = 9 - i # 회차 계산
    merge_col = "스트릿 우먼 파이터 " + str(episode) + "회"
    dic = {"회차" : episode,
           "시청률" : viewr_ratings[i],
           "회차_merge용" : merge_col}
    
    streetwomanfiger_viewr = streetwomanfiger_viewr.append(dic, ignore_index=True)

streetwomanfiger_viewr

,회차,시청률,회차_merge용
0,9,2.523%,스트릿 우먼 파이터 9회
1,8,2.887%,스트릿 우먼 파이터 8회
2,7,2.487%,스트릿 우먼 파이터 7회
3,6,2.672%,스트릿 우먼 파이터 6회
4,5,2.424%,스트릿 우먼 파이터 5회
5,4,2.593%,스트릿 우먼 파이터 4회
6,3,1.871%,스트릿 우먼 파이터 3회
7,2,1.897%,스트릿 우먼 파이터 2회
8,1,0.822%,스트릿 우먼 파이터 1회


In [20]:
# 시청률 csv 파일로 저장
streetwomanfiger_viewr.to_csv('streetwomanfiger_viewr.csv', index=False, encoding="utf-8-sig")

In [21]:
# 네이버 tv 크롤링 결과 가져오기
streetwomanfiger_view = pd.read_csv('streetwomanfiger_view.csv')
streetwomanfiger_view.rename(columns = {"회차" : "회"}, inplace=True)
streetwomanfiger_view

,회,영상 이름,조회수,좋아요 수
0,스트릿 우먼 파이터 9회,[스우파] 파이널리스트 무대 모아보기 | 라치카..,43232,197
1,스트릿 우먼 파이터 9회,[스우파] 파이널리스트 무대 모아보기 | 코카N..,11944,151
2,스트릿 우먼 파이터 9회,[스우파] 파이널리스트 무대 모아보기 | 홀리뱅..,27211,791
3,스트릿 우먼 파이터 9회,[스우파] 파이널리스트 무대 모아보기 | 훅 (HO..,28086,578
4,스트릿 우먼 파이터 9회,[9회 예고] 반가운 노제&효진초이★ 오랜만에 모..,49241,285
...,...,...,...,...
492,스트릿 우먼 파이터 1회,[1회/리액션캠] Fight Judge 보아x태용x황상훈 ..,8532,165
493,스트릿 우먼 파이터 1회,[스우파/1회 하이라이트] '약자 지목 배틀' 텐..,23899,224
494,스트릿 우먼 파이터 1회,[1회/풀캠] YGX 리정 vs 원트 이채연 @약자 지..,17275,129
495,스트릿 우먼 파이터 1회,[1회/리액션캠] MC 강다니엘 | YGX 리정 vs 원..,11481,869


In [22]:
streetwomanfiger = pd.merge(streetwomanfiger_viewr, streetwomanfiger_view, left_on='회차_merge용', right_on='회')
streetwomanfiger = streetwomanfiger.drop(['회차_merge용', '회'], axis=1)
streetwomanfiger

,회차,시청률,영상 이름,조회수,좋아요 수
0,9,2.523%,[스우파] 파이널리스트 무대 모아보기 | 라치카..,43232,197
1,9,2.523%,[스우파] 파이널리스트 무대 모아보기 | 코카N..,11944,151
2,9,2.523%,[스우파] 파이널리스트 무대 모아보기 | 홀리뱅..,27211,791
3,9,2.523%,[스우파] 파이널리스트 무대 모아보기 | 훅 (HO..,28086,578
4,9,2.523%,[9회 예고] 반가운 노제&효진초이★ 오랜만에 모..,49241,285
...,...,...,...,...,...
492,1,0.822%,[1회/리액션캠] Fight Judge 보아x태용x황상훈 ..,8532,165
493,1,0.822%,[스우파/1회 하이라이트] '약자 지목 배틀' 텐..,23899,224
494,1,0.822%,[1회/풀캠] YGX 리정 vs 원트 이채연 @약자 지..,17275,129
495,1,0.822%,[1회/리액션캠] MC 강다니엘 | YGX 리정 vs 원..,11481,869
